In [ ]:
import os
import numpy as np
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd

image_dir = r'path to your nifitimages'
mask_dir = r'path to your nifitmasks'
output_dir = r'Path to save files'
output_path = os.path.join(output_dir, 'hematoma_volumes.xlsx')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

image_files = {f: os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.nii.gz')}
mask_files = {f: os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.nii.gz')}

results = []
unmatched_files = []

def resample_image(image, reference):
    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(reference)
    resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetTransform(sitk.Transform())
    resample.SetOutputDirection(reference.GetDirection())
    resample.SetOutputOrigin(reference.GetOrigin())
    resample.SetOutputSpacing(reference.GetSpacing())
    resample.SetDefaultPixelValue(0)
    return resample.Execute(image)

def convert_vector_to_scalar(image):
    if image.GetNumberOfComponentsPerPixel() > 1:
        selector = sitk.VectorIndexSelectionCastImageFilter()
        selector.SetIndex(0)
        image = selector.Execute(image)
    return image

for image_name, image_path in image_files.items():
    if image_name in mask_files:
        mask_path = mask_files[image_name]
        print(f"Processing sample: {image_name}")
        
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)
        mask = resample_image(mask, image)
        image = convert_vector_to_scalar(image)
        image = sitk.Cast(image, sitk.sitkFloat32)
        mask = sitk.Cast(mask, sitk.sitkUInt8)
        
        params = {
            'geometryTolerance': 1e-4,
            'resampledPixelSpacing': None,
            'interpolator': sitk.sitkBSpline,
        }
        
        extractor = featureextractor.RadiomicsFeatureExtractor(**params)
        extractor.enableFeatureClassByName('shape')
        
        result = extractor.execute(image, mask)
        
        voxel_volume = result.get('original_shape_VoxelVolume', 'N/A')
        mesh_volume_mm3 = result.get('original_shape_MeshVolume', 'N/A')
        
        mesh_volume_ml = round(mesh_volume_mm3 / 1000, 3) if mesh_volume_mm3 != 'N/A' else 'N/A'
        
        results.append({
            'Image File': image_name,
            'Voxel Volume (number of voxels)': voxel_volume,
            'Mesh Volume (mm^3)': mesh_volume_mm3,
            'Mesh Volume (mL)': mesh_volume_ml
        })
    else:
        unmatched_files.append(image_name)

df = pd.DataFrame(results)
df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

if unmatched_files:
    print("The following image files did not find matching mask files or encountered errors during processing:")
    for file in unmatched_files:
        print(file)
else:
    print("All image files have been successfully matched with mask files.")